# **Import**

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

!pip install transformers
!pip install -U datasets

from collections import defaultdict, Counter
import json
import numpy as np
import torch


from matplotlib import pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader

Mounted at /content/gdrive


In [ ]:
# Read in hybrid dataset containing OpenAI summaries of false rows with the original dataset
banktrack_df = pd.read_csv('/content/gdrive/MyDrive/Group 1: DSSI Summer 2025/Data/summary_banktrak.csv')

# Convert labels to boolean values
banktrack_df["contains_debt_instrument_information"] = banktrack_df["contains_debt_instrument_information"].astype(int)
banktrack_df.head()

,text,company,cik,item,contains_debt_instrument_information
0,"Senior Secured Credit Facility On May 3, ...",LIEFLOGGER TECHNOLOGIES CORP.,1567771,1.01,1
1,"On January 9, 2019, Lifelogger Technologies...",LIEFLOGGER TECHNOLOGIES CORP.,1567771,1.01,1
2,Promissory Note and Warrants to SBI Investm...,LIEFLOGGER TECHNOLOGIES CORP.,1567771,1.01,1
3,"On March 1, 2016, Lifelogger Technologies C...",LIEFLOGGER TECHNOLOGIES CORP.,1567771,1.01,1
4,Acquisition of Pixorial Assets In further...,LIEFLOGGER TECHNOLOGIES CORP.,1567771,1.01,1


Alternatively, you can read in the code with the cell below

In [ ]:
#from datasets import load_dataset

# dataset = load_dataset('csv', data_files='/content/gdrive/MyDrive/Group 1: DSSI Summer 2025/Data/summary_banktrak.csv')
# dataset

In [ ]:
from datasets import Dataset, DatasetDict

# Convert DataFrame to a dataset dictionary
dataset = DatasetDict({
    'train': Dataset.from_pandas(banktrack_df)
})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'company', 'cik', 'item', 'contains_debt_instrument_information'],
        num_rows: 220
    })
})

**If using the original dataset, uncomment below to undersample majority label or use RandomUnderSampler**

In [ ]:
# from datasets import DatasetDict
# import numpy as np

# def undersample_majority_class(dataset_dict, label_column):
#     # Extract train split
#     np.random.seed(42)

#     train_data = dataset_dict["train"]
#     labels = np.array(train_data[label_column])

#     # Identify majority and minority classes
#     class_counts = np.bincount(labels)
#     minority_class = np.argmin(class_counts)
#     minority_count = np.min(class_counts)

#     # Get indices for both classes
#     minority_indices = np.where(labels == minority_class)[0]
#     majority_indices = np.where(labels != minority_class)[0]

#     # Randomly select subset of majority class to match minority count
#     undersampled_majority_indices = np.random.choice(
#         majority_indices,
#         size=minority_count,  # Match minority count
#         replace=False  # No duplicates
#     )

#     # Combine selected majority + all minority samples
#     new_indices = np.concatenate([undersampled_majority_indices, minority_indices])

#     # Create new balanced dataset
#     balanced_train = train_data.select(new_indices)

#     return DatasetDict({
#         "train": balanced_train,
#         # "validation": dataset_dict["validation"]  # Validation remains unchanged
#     })

# # Usage
# dataset = undersample_majority_class(
#     dataset,
#     label_column="contains_debt_instrument_information"
# )

# # Verify
# print("Balanced counts:", np.bincount(dataset["train"]["contains_debt_instrument_information"]))


# **Split Dataset into Training and Validation**

In [ ]:
#S plit dataset to training and validation
split_dataset = dataset["train"].train_test_split(
    test_size=0.2,      # 20% for validation
    seed=42,            # For reproducibility
    shuffle=True        # Shuffle before splitting
)

# Create the final DatasetDict with both splits
dataset = DatasetDict({
    "train": split_dataset["train"],          # 80%
    "validation": split_dataset["test"]       # 20%
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'company', 'cik', 'item', 'contains_debt_instrument_information'],
        num_rows: 176
    })
    validation: Dataset({
        features: ['text', 'company', 'cik', 'item', 'contains_debt_instrument_information'],
        num_rows: 44
    })
})


# **Dataset Preprocessing**

In [ ]:
from transformers import AutoTokenizer

name = "typeform/distilroberta-base-v2"

# Initialize your tokenizer here
tokenizer = AutoTokenizer.from_pretrained(name)

sample_input = "We want to use a pretrained tokenizer."

# Call your tokenizer here to check if it was properly loaded by using on a test sentence
tokenized_inputs = tokenizer(
    sample_input,
    padding = "max_length",
    truncation= True,
    max_length = 64,
    return_tensors = "pt"
)
print(tokenized_inputs["input_ids"])

tensor([[    0,   170,   236,     7,   304,    10, 11857, 26492, 19233,  6315,
             4,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1]])


In [ ]:
# Now that our tokenizer has been properly loaded, we need to call the tokenizer
# for every example in the dataset. Here we use list comprehension with a
# lambda function ensure that.
tokenized_dataset = dataset.map(
    lambda example: tokenizer(example['text'], padding="max_length",
    truncation=True, max_length=250)
)

# We need to remove these extra columns before the dataset can be sent to the
# dataloader and subsequently to the model. Also be sure to check that the
# output column is named labels or else rename if necessary
tokenized_dataset = tokenized_dataset.remove_columns(['item', 'text', 'company','cik'])
tokenized_dataset = tokenized_dataset.rename_column("contains_debt_instrument_information", "labels")

tokenized_dataset.set_format("torch")

Map:   0%|          | 0/176 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

# **Training and Validation**

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "/content/gdrive/MyDrive/DSSI/distilroberta-model"

model = AutoModelForSequenceClassification.from_pretrained("typeform/distilroberta-base-v2", num_labels=2).to(device)

## Freeze embedding layers
# for param in model.roberta.embeddings.parameters():
#     param.requires_grad = False

arguments = TrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=3e-5,
    logging_dir="./logs",
    logging_steps=10,
    metric_for_best_model="eval_loss",
    greater_is_better=True,
    load_best_model_at_end=True,
    seed=42
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    # calculates the accuracy
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# train the model
trainer.train()

# **Evaluation**

In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
results = trainer.predict(tokenized_dataset['validation']) # also gives you predictions
results

In [ ]:
test_logits, y_true, metrics = results
print(len(test_logits), len(y_true))

#Convert the logits to predicted labels : if this a torch tensor use torch.argmax(test_logits, dim = 1)
y_pred = np.argmax(test_logits, axis = 1)
print(y_true[:10])
print(y_pred[:10])

#sanity check: should have as many predictions as labels
assert len(y_pred)==len(y_true)


In [ ]:
print('F1 Score:',f1_score(y_true, y_pred))
print('Accuracy Score:',accuracy_score(y_true, y_pred))

# **Error Analysis**

In [ ]:
import pandas as pd

# Create a DataFrame to store texts, true labels, and predictions
misclassified = []
for i, (true, pred) in enumerate(zip(y_true, y_pred)):
    if true != pred:
        misclassified.append({
            "text": dataset["validation"][i]["text"],
            "true_label": true,
            "predicted_label": pred,
            "logits": test_logits[i]
        })

# Convert to DataFrame
misclassified_df = pd.DataFrame(misclassified)
print(f"Number of misclassified examples: {len(misclassified_df)}")
misclassified_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Example data (replace with your actual predictions and true labels)


# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Print the raw confusion matrix
print("Confusion Matrix:")
print(cm)

# Visualize with matplotlib/seaborn
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# Separate FP and FN
fp = misclassified_df[misclassified_df["true_label"] == 0]
fn = misclassified_df[misclassified_df["true_label"] == 1]

print(f"False Positives (FP): {len(fp)}")
print(f"False Negatives (FN): {len(fn)}")

# Inspect FP
print("\nFalse Positives (FP):")
for idx, row in fp.iterrows():
    print(f"Text: {row['text'][:]}...")  # Print first 200 chars
    print(f"True: {row['true_label']}, Predicted: {row['predicted_label']}\n")

# Inspect FN
print("\nFalse Negatives (FN):")
for idx, row in fn.iterrows():
    print(f"Text: {row['text'][:]}...")
    print(f"True: {row['true_label']}, Predicted: {row['predicted_label']}\n")

In [ ]:
from wordcloud import WordCloud

# Combine all FP/FN texts
fp_text = " ".join(fp['text'])  # Replace with your FP texts
fn_text = " ".join(fn['text'])  # Replace with your FN texts

# Generate word clouds
wordcloud_fp = WordCloud(width=800, height=400, background_color='white').generate(fp_text)
wordcloud_fn = WordCloud(width=800, height=400, background_color='white').generate(fn_text)

# Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_fp, interpolation='bilinear')
plt.title('False Positives')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(wordcloud_fn, interpolation='bilinear')
plt.title('False Negatives')
plt.axis('off')
plt.show()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import matplotlib.pyplot as plt
from collections import Counter


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return words

# Example FP and FN texts (replace with your data)
fp_texts = [
    "Underwriting Agreement  On September 30, 2015, Concho Resources Inc. (the  Company ) entered into an underwriting agreement (the  Underwriting Agreement ) with J.P. Morgan Securities LLC and Credit Suisse Securities (USA), LLC, as representatives (the  Representatives ) of the several underwriters named therein (the  Underwriters ), in connection with an underwritten public offering of up to 8,855,000 shares of common stock (the  Common Shares ), including the Underwriters option to purchase up to an  additional 1,155,000 Common Shares. On October 1, 2015, the Underwriters exercised in full their option to purchase additional Common Shares. The Company expects the net proceeds from the offering to be approximately $794 million, after deducting  estimated fees and expenses. Net proceeds from the offering are expected to be used to repay all outstanding borrowings under the Companys credit facility, which were used in part to finance recent acquisitions, and for general corporate  purposes, including funding potential future acquisitions. The issuance and sale of the Common Shares has been registered under the Securities Act of 1933, as amended (the  Securities Act ), pursuant to an automatic shelf  Registration Statement on Form S-3 (Registration No. 333-206172) of the Company, filed with and deemed automatically effective by the Securities and Exchange Commission on August 6, 2015. Closing of the issuance and sale of the Common  Shares is scheduled for October 6, 2015, subject to customary closing conditions. A legal opinion relating to the validity of the Common Shares is filed herewith as Exhibit 5.1.  The Underwriting Agreement contains customary representations, warranties and agreements by the Company and customary conditions to closing,  obligations of the parties and termination provisions. Additionally, the Company has agreed to indemnify the Underwriters against certain liabilities, including liabilities under the Securities Act, or to contribute to payments the Underwriters may  be required to make because of any of those liabilities. Furthermore, the Company has agreed with the Underwriters not to offer or sell any shares of its common stock (or securities convertible into or exchangeable for common stock), subject to  limited exceptions, for a period of 60 days after the date of the Underwriting Agreement without the prior written consent of the Representatives.  Certain of the Underwriters and their affiliates act as administrative agent, lender, swingline lender and/or letter of credit lender under  the Companys credit facility. Further, certain of the Underwriters or their affiliates have from time to time provided investment banking, commercial banking and financial advisory services to the Company, for which they have received  customary compensation. The Underwriters and their affiliates may provide similar services in the future. In addition, from time to time, certain of the Underwriters and their affiliates may effect transactions for their own account or the account  of customers, and hold on behalf of themselves or their customers, long or short positions in the Companys debt or equity securities or loans, and may do so in the future.  The foregoing description of the Underwriting Agreement is qualified in its entirety by reference to such Underwriting Agreement, a copy of  which is filed herewith as Exhibit 1.1 and is incorporated herein by reference...."

]

fn_texts = [
    "In anticipation of consummating the transactions described in Item 1.01 of this Current Report on Form 8-K, on July 15, 2021, Enstar borrowed $300 million under the Revolving Credit Facility, the proceeds of which will be used to fund in part the purchase under the EGL Agreement and pay costs and expenses associated therewith. Following the draw down, Enstar has approximately $125 million of undrawn borrowing capacity under the Revolving Credit Facility. A summary of the material terms of the Revolving Credit Facility is included in Enstar's Annual Report on  Form 10-K  for the year ended December 31, 2020 filed with the SEC on March 1, 2021 and is incorporated herein by reference...",
     "On August 19, 2020, Enstar Group Limited ('the Company') and its wholly-owned subsidiary, Enstar Finance LLC ('Enstar Finance'), entered into an Underwriting Agreement (the 'Underwriting Agreement') with Wells Fargo Securities, LLC, Barclays Capital Inc., HSBC Securities (USA) Inc. and J.P. Morgan Securities LLC, as representatives of the several underwriters named therein, for the issuance and sale of $350 million aggregate principal amount of 5.75% Fixed-Rate Reset Junior Subordinated Notes due 2040 (the 'Junior Subordinated Notes'). The Junior Subordinated Notes will be issued by Enstar Finance and fully and unconditionally guaranteed, on a junior subordinated basis, by the Company. The sale of the Junior Subordinated Notes has been registered under the Securities Act of 1933, as amended (the 'Securities Act'), pursuant to an automatic shelf registration statement (the 'Registration Statement') on Form S-3 (File Nos. 333-247995 and 333-247995-01) filed with the Securities and Exchange Commission on August 17, 2020. The Underwriting Agreement contains customary representations, warranties and agreements by the Company and Enstar Finance, customary conditions to closing, other obligations of the parties and termination provisions. Additionally, the Company and Enstar Finance have agreed to indemnify the underwriters against certain liabilities, including liabilities under the Securities Act, or to contribute to payments the underwriters may be required to make because of any of those liabilities. The foregoing description of the Underwriting Agreement is qualified by reference to the agreement itself, which is attached as Exhibit 1.1 to this report...."


]

# Preprocess all FP and FN texts
fp_words = []
for text in fp_texts:
    fp_words.extend(preprocess_text(text))

fn_words = []
for text in fn_texts:
    fn_words.extend(preprocess_text(text))

In [ ]:
# Count word frequencies
fp_word_counts = Counter(fp_words)
fn_word_counts = Counter(fn_words)

# Get top N most common words
top_n = 20
top_fp_words = fp_word_counts.most_common(top_n)
top_fn_words = fn_word_counts.most_common(top_n)

# Separate words and counts for plotting
fp_words, fp_counts = zip(*top_fp_words)
fn_words, fn_counts = zip(*top_fn_words)

In [ ]:
# Plot FP word frequencies
plt.figure(figsize=(12, 6))
plt.bar(fp_words, fp_counts, color='red')
plt.title('Top 20 Words in False Positives (FP)')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

# Plot FN word frequencies
plt.figure(figsize=(12, 6))
plt.bar(fn_words, fn_counts, color='blue')
plt.title('Top 20 Words in False Negatives (FN)')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import numpy as np

# Combine FP and FN data
words_combined = list(set(fp_words).union(set(fn_words)))
fp_combined_counts = [fp_word_counts.get(word, 0) for word in words_combined]
fn_combined_counts = [fn_word_counts.get(word, 0) for word in words_combined]

# Sort by FP counts (descending)
sorted_indices = np.argsort(fp_combined_counts)[::-1]
words_combined = [words_combined[i] for i in sorted_indices]
fp_combined_counts = [fp_combined_counts[i] for i in sorted_indices]
fn_combined_counts = [fn_combined_counts[i] for i in sorted_indices]

# Plot
plt.figure(figsize=(14, 7))
x = np.arange(len(words_combined))
width = 0.35

plt.bar(x - width/2, fp_combined_counts, width, label='False Positives', color='red')
plt.bar(x + width/2, fn_combined_counts, width, label='False Negatives', color='blue')

plt.title('Word Frequencies in Misclassified Samples')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(x, words_combined, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()